# Script to convert netCDF climate files from the Met Office Unified Model into PSG GCM files compativle with the GlobES tool. Created by Fauchez, Villanueva - NASA Goddard Space Flight Center, March 2021. Scripts amended to also include 3-D Chemistry variables and eccentric orbit (M. Braam, CSH, University of Bern), Oct 2024

In [1]:
# ---------------------------------------------------------------
# Script to convert netCDF climate files into PSG GCM files
# netCDF files in UM general circulation model
# Fauchez, Villanueva - NASA Goddard Space Flight Center
# March 2021
# Amended to also include 3-D Chemistry variables (M. Braam, CSH, University of Bern)
# ---------------------------------------------------------------
import sys, os
import numpy as np
from netCDF4 import Dataset as ncdf

# Module that performs the conversion
def convertgcm(filein = 'data/gcm_um.nc', fileout = 'PSG/GlobES/spectra/spectra0224/gcm_psg_pcb_11_i70_12_w.dat', 
               itime=12, sol_lon=0.0):
	# Read the netCDF file
	nfile = ncdf(filein)
#	print(nfile.variables)
	lat = (nfile.variables['latitude'])[:];
	lon = (nfile.variables['longitude'])[:];
	alt = (nfile.variables['hybrid_ht'])[:];
	tsurf = (nfile.variables['temp'])[:]; tsurf = tsurf[itime,:,:]
	temp = (nfile.variables['temp_1'])[:];  temp = temp[itime,:,:,:]
	press = (nfile.variables['p'])[:]; press = press[itime,:,:,:]
	uk = (nfile.variables['u'])[:];    uk = uk[itime,:,:,:]
	vk = (nfile.variables['v'])[:];    vk = vk[itime,:,:,:]
	h2o = (nfile.variables['q'])[:];   h2o = h2o[itime,:,:,:] * 28./18.
	ice = (nfile.variables['QCF'])[:];   ice = ice[itime,:,:,:]
	liq = (nfile.variables['QCL'])[:];   liq = liq[itime,:,:,:]
	o3 = (nfile.variables['field2101'])[:];   o3 = o3[itime,:,:,:] *28./48
	no = (nfile.variables['field2102'])[:];   no = no[itime,:,:,:] *28./30.01
	no2 = (nfile.variables['field3102'])[:];   no2 = no2[itime,:,:,:] *28./46
#	n2o = (nfile.variables['field2149'])[:];   n2o = n2o[itime,:,:,:] *28./44
#	hno3 = (nfile.variables['field2107'])[:];   hno3 = hno3[itime,:,:,:] *28./63
	print(nfile.variables['QCF'])
   
#	rliq = (nfile.variables['STASH_m01s01i221'])[:];  rliq = rliq[itime,:,:,:]/1e6# Size of ice clouds in m
	nfile.close()
	# Fix variables
	sz = np.shape(temp)
	print(sz)
	p = np.zeros(sz); pw = p[:-1,:,:]
	h2ow = np.zeros(sz); h2ow = h2o[:-1,:,:]
	icew = np.zeros(sz); icew = ice[:-1,:,:]
	liqw = np.zeros(sz); liqw = liq[:-1,:,:]
#	uw = np.zeros(sz); uw[:-1,:,:] = uk
	vw = np.zeros(sz); vw[:,:,:] = vk[:,:-1,:]
#	print(vw)
	tsurf = np.where((tsurf>0) & (np.isfinite(tsurf)), tsurf, 300.0)
	temp = np.where((temp>0) & (np.isfinite(temp)), temp, 300.0)
	h2ow = np.where((h2ow>0) & (np.isfinite(h2ow)), h2ow, 1e-30)
	icew = np.where((icew>0) & (np.isfinite(icew)), icew, 1e-30)
	liqw = np.where((liqw>0) & (np.isfinite(liqw)), liqw, 1e-30)
#	rliq = np.where((rliq>0) & (np.isfinite(rliq)), rliq, 1e-6)
	o3 = np.where((o3>0) & (np.isfinite(o3)), o3, 1e-30)
	no = np.where((no>0) & (np.isfinite(no)), no, 1e-30)
	no2 = np.where((no2>0) & (np.isfinite(no2)), no2, 1e-30)
#	n2o = np.where((n2o>0) & (np.isfinite(n2o)), n2o, 1e-30)
#	hno3 = np.where((hno3>0) & (np.isfinite(hno3)), hno3, 1e-30)
#
	# Save object parameters
	newf = []
	newf.append('<OBJECT>Exoplanet')
	newf.append('<OBJECT-NAME>Proxima Cen b')
	newf.append('<OBJECT-DIAMETER>14320')
	newf.append('<OBJECT-GRAVITY>10.9')
	newf.append('<OBJECT-GRAVITY-UNIT>g')
	newf.append('<OBJECT-STAR-DISTANCE>0.04850')
	newf.append('<OBJECT-STAR-VELOCITY>0.00138')
	newf.append('<OBJECT-SOLAR-LONGITUDE>%s' %sol_lon)
#	newf.append('<OBJECT-SOLAR-LATITUDE>0.0')
#	newf.append('<OBJECT-SEASON>0.0')
	newf.append('<OBJECT-STAR-TYPE>M')
	newf.append('<OBJECT-STAR-TEMPERATURE>3050')
	newf.append('<OBJECT-STAR-RADIUS>0.14')
	newf.append('<OBJECT-STAR-METALLICITY>0.0')
#	newf.append('<OBJECT-OBS-LONGITUDE>0.0')
#	newf.append('<OBJECT-OBS-LATITUDE>0.0')#20?
#	newf.append('<OBJECT-PERIOD>11.186')
	newf.append('<OBJECT-OBS-VELOCITY>-21.100')
	newf.append('<OBJECT-INCLINATION>70')
	newf.append('<OBJECT-ECCENTRICITY>0')#0.3 if eccentric orbit
	newf.append('<OBJECT-PERIAPSIS>103.00')
#	newf.append('<OBJECT-ORBIT>0.06300000,0.00000,1.58040433,0.00000,0.04544,2455701.776757')
#	newf.append('<OBJECT-POSITION-ANGLE>0.00')
	newf.append('<GEOMETRY>Observatory')
	newf.append('<GEOMETRY-OFFSET-NS>0.0')
	newf.append('<GEOMETRY-OFFSET-EW>0.0')
	newf.append('<GEOMETRY-OFFSET-UNIT>arcsec')
	newf.append('<GEOMETRY-OBS-ALTITUDE>1.3012')
	newf.append('<GEOMETRY-ALTITUDE-UNIT>pc')


	# Save atmosphere parameters
	newf.append('<ATMOSPHERE-DESCRIPTION>Met Office Unified Model (UM) simulation')
	newf.append('<ATMOSPHERE-STRUCTURE>Equilibrium')
	newf.append('<ATMOSPHERE-PRESSURE>100000')
	newf.append('<ATMOSPHERE-PUNIT>Pa')
	newf.append('<ATMOSPHERE-WEIGHT>28.0')
	newf.append('<ATMOSPHERE-LAYERS>60')
	newf.append('<ATMOSPHERE-NGAS>9')
	newf.append('<ATMOSPHERE-GAS>N2,O2,CO2,H2O,O3,NO,NO2,N2O,HNO3')
	newf.append('<ATMOSPHERE-TYPE>HIT[22],HIT[7],HIT[2],HIT[1],HIT[3],HIT[8],HIT[10],HIT[4],HIT[12]')
	newf.append('<ATMOSPHERE-ABUN>78,21,400,1,1,1,1,1,1')
	newf.append('<ATMOSPHERE-UNIT>pct,pct,ppm,scl,scl,scl,scl,scl,scl')
	newf.append('<ATMOSPHERE-NMAX>2')
	newf.append('<ATMOSPHERE-LMAX>2')
	newf.append('<ATMOSPHERE-NAERO>2')
	newf.append('<ATMOSPHERE-AEROS>Water,WaterIce')
	newf.append('<ATMOSPHERE-ATYPE>AFCRL_Water_HRI,Warren_ice_HRI')
	newf.append('<ATMOSPHERE-AABUN>1,1')
	newf.append('<ATMOSPHERE-AUNIT>scl,scl')
	newf.append('<ATMOSPHERE-ASIZE>5,100')
	newf.append('<ATMOSPHERE-ASUNI>um,um')

	# Save surface parameters
	newf.append('<SURFACE-TEMPERATURE>300')
	newf.append('<SURFACE-ALBEDO>0.2')
	newf.append('<SURFACE-EMISSIVITY>1.0')
	newf.append('<SURFACE-NSURF>0')

	# Save GCM parameters
	vars = '<ATMOSPHERE-GCM-PARAMETERS>'
	vars = vars + str("%d,%d,%d,%.1f,%.1f,%.2f,%.2f" %(sz[2],sz[1],sz[0],lon[0],lat[0],lon[1]-lon[0],lat[1]-lat[0]))
	vars = vars + ',Winds,Tsurf,Temperature,Pressure,H2O,Water,WaterIce,O3,NO,NO2'
	newf.append(vars)
	with open(fileout,'w') as fw:
		for i in newf: fw.write(i+'\n')
	with open(fileout,'ab') as fb:
		if sys.version_info>=(3,0,0): bc=fb.write(bytes('<BINARY>',encoding = 'utf-8'))
		else: bc=fb.write('<BINARY>')
		fb.write(np.asarray(uk,order='C',dtype=np.float32))
		fb.write(np.asarray(vw,order='C',dtype=np.float32))
		fb.write(np.asarray(tsurf,order='C',dtype=np.float32))
		fb.write(np.asarray(temp,order='C',dtype=np.float32))
		fb.write(np.log10(np.asarray(press*1e-5,order='C',dtype=np.float32)))
		fb.write(np.log10(np.asarray(h2ow,order='C',dtype=np.float32)))
		fb.write(np.log10(np.asarray(liqw,order='C',dtype=np.float32)))
		fb.write(np.log10(np.asarray(icew,order='C',dtype=np.float32)))
#		fb.write(np.log10(np.asarray(rliq,order='C',dtype=np.float32)))
		fb.write(np.log10(np.asarray(o3,order='C',dtype=np.float32)))
		fb.write(np.log10(np.asarray(no,order='C',dtype=np.float32)))
		fb.write(np.log10(np.asarray(no2,order='C',dtype=np.float32)))
#		fb.write(np.log10(np.asarray(n2o,order='C',dtype=np.float32)))
#		fb.write(np.log10(np.asarray(hno3,order='C',dtype=np.float32)))
		if sys.version_info>=(3,0,0): bc=fb.write(bytes('</BINARY>',encoding = 'utf-8'))
		else: bc=fb.write('</BINARY>')
	fb.close()
#End convert



# Defining orbital phase angles for Proxima Centauri b and changes in solar longitude due to 3:2 spin-orbit resonance.

In [2]:
phase_list23 = []
sol_lon_list23 = []
for i in range(23):
    phase = 102.94 + 32.1831 * i
    sol_lon = 0 + 16.09 * i
    wrapped_phase = phase % 360
    wrapped_sol_lon = sol_lon % 360
    phase_list23.append(wrapped_phase)
    sol_lon_list23.append(wrapped_sol_lon)
phase_list23

[102.94,
 135.1231,
 167.3062,
 199.48930000000001,
 231.6724,
 263.8555,
 296.03860000000003,
 328.22170000000006,
 0.4048000000000229,
 32.58790000000005,
 64.77100000000002,
 96.95410000000004,
 129.13720000000006,
 161.3203000000001,
 193.50340000000006,
 225.68650000000002,
 257.8696,
 290.05269999999996,
 322.23580000000015,
 354.4189000000001,
 26.60200000000009,
 58.78510000000006,
 90.96820000000002]

In [2]:
#Non-eccentric, 1:1 spin-orbit resonance
for i in range(0,13):
    convertgcm(filein='data/pcb_11_daily_7458_1.nc', 
       fileout = 'PSG/GlobES/spectra/spectra0224/gcm_psg_pcb_11_i70_%s_w.dat' %i, itime=i) 
#Eccentric, 3:2 spin-orbit resonance
for i in range(0,13):
#     convertgcm(filein='../eccent_paper/data/pcb_32_fix_17120_120.nc', 
#                fileout = 'PSG/GlobES/spectra/spectra0224/gcm_psg_pcb_32_i70_fix_%s_w.dat' %i, 
#                itime=i, sol_lon=sol_lon_list23[i]) 

<class 'netCDF4._netCDF4.Variable'>
float32 QCF(t, hybrid_ht_2, latitude, longitude)
    source: Unified Model Output (Vn12.0):
    name: QCF
    title: QCF AFTER TIMESTEP
    date: 25/09/24
    time: 00:00
    long_name: QCF AFTER TIMESTEP
    standard_name: mass_fraction_of_cloud_ice_in_air
    units: kg kg-1
    missing_value: 2e+20
    _FillValue: 2e+20
    valid_min: 0.0
    valid_max: 0.00077956624
unlimited dimensions: t
current shape = (120, 61, 90, 144)
filling on
(60, 90, 144)


# Using the PSG API to feed the GCM files along with orbital information to PSG to produce synthetic emission spectra focused on the spectral range of the Large Interferometer For Exoplanets
## Folders need to be changed to own directories, example config.txt file provided

In [4]:
import matplotlib.pyplot as plt

In [6]:
psgurl = 'https://psg.gsfc.nasa.gov' # URL of the PSG server


In [8]:
# Shown for 1:1 spin-orbit resonance, change names for 3:2 spin-orbit resonance
for i in range(0,13):
    update = True
    phase = phase_list23[i]       # Orbital phases
    binning= 3        # Binning applied to the GCM data for each radiative-transfer (greater is faster, minimum is 1)
    lam1   = 4         # Initial wavelength of the simulations (um)
    lam2   = 20.0        # Final wavelength of the simulations (um)
    lamRP  = 0.0654      # Resolving power
    radunit= 'Wsrm2um'       # Desired radiation unit (https://psg.gsfc.nasa.gov/helpmodel.php#units)
    print(phase)
    # GlobES/API calls can be sequentially, and PSG will remember the previous values
    # This means that we can upload parameters step-by-step. To reset your config for GlobES (use type=set),
    # and to simply update (use type=upd)
    if update: os.system('curl -s -d app=globes -d type=set --data-urlencode file@PSG/GlobES/spectra/spectra0224/gcm_psg_pcb_11_i70_%s_w.dat %s/api.php' % (i,psgurl))

    # Define parameters of this run
    fr = open("PSG/GlobES/spectra/spectra0224/config.txt", "w")
    fr.write('<GENERATOR-RANGE1>%f\n' % lam1)
    fr.write('<GENERATOR-RANGE2>%f\n' % lam2)
    fr.write('<GENERATOR-RANGEUNIT>um\n')
    fr.write('<GENERATOR-RESOLUTION>%f\n' % lamRP)
    fr.write('<GENERATOR-RESOLUTIONUNIT>um\n')
    fr.write('<GENERATOR-RADUNITS>%s\n' % radunit)
    fr.write('<GENERATOR-GCM-BINNING>%d\n' % binning)
    fr.write('<OBJECT-INCLINATION>70\n')
    fr.write('<OBJECT-SOLAR-LATITUDE>0.0\n')
    fr.write('<OBJECT-OBS-LATITUDE>0.0\n')
    fr.write('<OBJECT-SEASON>%f\n' % phase)
    fr.write('<OBJECT-OBS-LONGITUDE>%f\n' % phase)
    fr.close()
#     if update: os.system('curl -s -d app=globes -d type=upd --data-urlencode file@PSG/GlobES/spectra/spectra0224/config.txt %s/api.php' % psgurl)
    if update: os.system('curl -s -d app=globes --data-urlencode file@PSG/GlobES/spectra/spectra0224/config.txt %s/api.php > PSG/GlobES/spectra/spectra0224/pcb_11_fix_phase%d.txt' % (psgurl,phase))
    data = np.genfromtxt('PSG/GlobES/spectra/spectra0224/pcb_11_fix_phase%d.txt' % int(phase))


102.94
